### Preprocessing  
Sofia, Angel

In [4]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# list of file name strings from S1 and S2
S1_files = sorted(glob.glob('../S1_Dataset/d1p*'))
S2_files = sorted(glob.glob('../S2_Dataset/d2p*'))

# TODO: when done with program, merge all data from S1 and S2 into 1 giant dataset
# for testing only using 1 file for now
file = S1_files[0]


df = pd.read_csv(file, header=None)
df.columns = [
    'time',
    'frontal accel',
    'vertical accel',
    'lateral accel',
    'antenna id',
    'rssi',
    'phase',
    'frequency',
    'activity',
]
activities = {
    1: 'sit on bed',
    2: 'sit on chair',
    3: 'lying',
    4: 'ambulating',
}
df.replace({'activity': activities}, inplace=True)

# TODO: add 3 columns activity status, frequency of activity type (time over total time), and gender 
# our target is "activity status" (inactive or active) with our independent variables being activity type, time, rssi, frontal accel, vertical accel, lateral accel, gender, frequency (of activity type)


# TODO: preprocess data and split into test and training sets using df
# normalize data 
# split 25:75 training and test
df

,time,frontal accel,vertical accel,lateral accel,antenna id,rssi,phase,frequency,activity
0,0.00,0.27203,1.00820,-0.082102,1,-63.5,2.42520,924.25,sit on bed
1,0.50,0.27203,1.00820,-0.082102,1,-63.0,4.73690,921.75,sit on bed
2,1.50,0.44791,0.91636,-0.013684,1,-63.5,3.03110,923.75,sit on bed
3,1.75,0.44791,0.91636,-0.013684,1,-63.0,2.03710,921.25,sit on bed
4,2.50,0.34238,0.96229,-0.059296,1,-63.5,5.89200,920.25,sit on bed
...,...,...,...,...,...,...,...,...,...
396,251.00,0.87003,0.46851,0.009122,4,-58.5,0.73631,921.75,sit on bed
397,251.25,0.87003,0.46851,0.009122,1,-57.5,0.70870,923.25,sit on bed
398,251.50,0.87003,0.46851,0.009122,1,-59.0,0.25157,924.75,sit on bed
399,253.25,-0.00938,0.95081,-0.116310,1,-58.5,0.35282,924.25,sit on bed


The code above wasn't working when I ran it on my computer. I'm not as experienced in jupyter notebook so maybe I'm doing something wrong? This version of the code should work if you're trying to extract the files from the zipped folder. -- Angel

In [67]:
from zipfile import ZipFile, ZipInfo
import pandas as pd

## this is the zipped folder name on my computer, idk if its the same for everyone
zipped_name = 'activity+recognition+with+healthy+older+people+using+a+batteryless+wearable+sensor.zip'

## honestly, idk what this is doing, but i feel its important stuff
z = ZipFile(zipped_name, 'r')
ls1 = z.infolist()
ls2 = [file for file in ls1 if file.file_size > 0]


In [68]:
## im going to seperate the files into four categories to make adding extra attributes easier
S1F = []
S1M = []
S2F = []
S2M = []
## this one is just to check my work
other = [] #ignore

## i feel like there was a more efficient way to do this
## but this segment of code extracts the content of the files
 # and converts them into pandas dataframes
for i in range(len(ls2)):
    temp_zip = z.extract(member=ls2[i])
    if (("S1_Dataset" in temp_zip) and ("F" in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S1F.append(temp_csv)
    elif (("S1_Dataset" in temp_zip) and ('M' in temp_zip) and ('.txt' not in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S1M.append(temp_csv)
    elif (("S2_Dataset" in temp_zip) and ("F" in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S2F.append(temp_csv)
    elif (("S2_Dataset" in temp_zip) and ("M" in temp_zip) and ('.txt' not in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S2M.append(temp_csv)
    else:
        other.append(temp_zip)

In [69]:
## all dataframes in one list, seperated into four categories
 # (based on gender and room)
all_datasets = [S1F, S1M, S2F, S2M]

features = [
    'time',
    'frontal accel',
    'vertical accel',
    'lateral accel',
    'antenna id',
    'rssi',
    'phase',
    'frequency',
    'activity',
]

activities = [
    'sitting on bed',
    'sitting on chair',
    'lying',
    'ambulating'
]

## in this segment of code, i rename the headers
new_headers = dict(enumerate(features))
new_activities = dict(enumerate(activities, start=1))
for subset in all_datasets:
    for df in subset:
        df.rename(columns = new_headers, inplace= True)
        df['activity'] = df['activity'].replace(new_activities)


In [70]:
##  CAN ONLY RUN ONCE APPARENTLY
##

## adding F/M and room columns to dataframes 
for df in S1F:
    df.insert(len(df.columns) - 1, 'gender', 'F')
    df.insert(1, 'room', 'S1')

for df in S1M:
    df.insert(len(df.columns) - 1, 'gender', 'M')
    df.insert(1, 'room', 'S1')

for df in S2F:
    df.insert(len(df.columns) - 1, 'gender', 'F')
    df.insert(1, 'room', 'S2')

for df in S2M:
    df.insert(len(df.columns) - 1, 'gender', 'M')
    df.insert(1, 'room', 'S2')


We can go through each dataframe and add additional columns now. In these next cells, I add the 'consecutiveness' feature. This tells us how many times in a row an individual did an activity. 

For example, the first observation will have a 1 in this column. If the individual continued to do the same activity in the next observation, that obervation will have a 2 for the 'consecutivness' feature.
Otherwise, the count will start over at 1.

In [71]:
## function to add in the 'consecutiveness' attribute in a dataframe
def count_activities(dataframe):
    counts = []
    curr_count = 0
    prev_act = None

    for act in df['activity']:
        if act == prev_act:
            curr_count += 1
        else:
            curr_count = 1
            prev_act = act
    
        counts.append(curr_count)

    dataframe.insert(len(dataframe.columns) - 1, 'consecutiveness', counts)
    return dataframe

list_of_dataframes = S1F + S1M + S2F + S2M

##   ONLY RUN ONCE
##

## adds the column 'consecutiveness' into each dataframe
for df in list_of_dataframes:
    count_activities(df)


In [72]:
## checking my work
from random import randint

list_of_dataframes[randint(0,len(list_of_dataframes))]

,time,room,frontal accel,vertical accel,lateral accel,antenna id,rssi,phase,frequency,gender,consecutiveness,activity
0,0.00,S1,0.27203,1.00820,-0.082102,1,-63.5,2.42520,924.25,M,1,sitting on bed
1,0.50,S1,0.27203,1.00820,-0.082102,1,-63.0,4.73690,921.75,M,2,sitting on bed
2,1.50,S1,0.44791,0.91636,-0.013684,1,-63.5,3.03110,923.75,M,3,sitting on bed
3,1.75,S1,0.44791,0.91636,-0.013684,1,-63.0,2.03710,921.25,M,4,sitting on bed
4,2.50,S1,0.34238,0.96229,-0.059296,1,-63.5,5.89200,920.25,M,5,sitting on bed
...,...,...,...,...,...,...,...,...,...,...,...,...
396,251.00,S1,0.87003,0.46851,0.009122,4,-58.5,0.73631,921.75,M,4,sitting on bed
397,251.25,S1,0.87003,0.46851,0.009122,1,-57.5,0.70870,923.25,M,5,sitting on bed
398,251.50,S1,0.87003,0.46851,0.009122,1,-59.0,0.25157,924.75,M,6,sitting on bed
399,253.25,S1,-0.00938,0.95081,-0.116310,1,-58.5,0.35282,924.25,M,7,sitting on bed


This next section of code adds the acceleration column.

In [75]:
import numpy as np 

def calc_and_add_accel(dataframe):
    acceleration = []
    for i in range(dataframe.shape[0]):
        fa = dataframe['frontal accel'][i]
        la = dataframe['lateral accel'][i]
        va = dataframe['vertical accel'][i]

        accel_vector = np.sqrt(fa**2 + la**2 + va**2)
        acceleration.append(accel_vector)

    dataframe['acceleration'] = acceleration
    return dataframe


In [76]:
## JUST RUN ONCE

for df in list_of_dataframes:
    calc_and_add_accel(df)

In [84]:
## checking my work again
from random import randint

list_of_dataframes[randint(0,len(list_of_dataframes))]

,time,room,frontal accel,vertical accel,lateral accel,antenna id,rssi,phase,frequency,gender,consecutiveness,activity,acceleration
0,0.00,S2,-0.091458,0.97377,0.20297,3,-61.0,0.87744,922.75,F,1,ambulating,0.998894
1,0.75,S2,0.483090,0.84746,-0.41279,3,-58.5,0.96641,924.75,F,1,sitting on bed,1.059226
2,1.75,S2,0.952100,0.42258,-0.35577,2,-51.5,6.03620,920.25,F,1,lying,1.100745
3,2.25,S2,0.975550,0.37665,-0.41279,3,-62.0,1.16740,921.75,F,2,lying,1.124259
4,3.00,S2,0.940380,0.13550,-0.42419,3,-68.5,5.57910,920.75,F,3,lying,1.040486
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,566.58,S2,1.081100,0.21588,-0.60664,2,-48.5,2.64000,920.25,F,65,lying,1.258330
136,566.75,S2,0.483090,0.60631,-0.85751,2,-54.0,2.35310,922.25,F,1,sitting on bed,1.155989
137,567.25,S2,0.471360,0.53741,-0.65225,3,-59.0,4.89030,923.25,F,2,sitting on bed,0.967688
138,567.75,S2,0.318930,0.91636,-0.37858,3,-58.5,5.35670,920.75,F,1,ambulating,1.041516


### Building + Testing Model

**Hyper param testing**  
Martin  


In [37]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor

# TODO: add hyperparameters that we will be testing as parameters for this function (Martin)
def create_model(): 
    print("NOT IMPLEMENTED")
    # TODO: implement our nueral network structure using SGD as the optimization technique (Martin)

    # maybe use tf.keras.sequential here?
    model = 0
    return model
    
# TODO: add hyper parameters that we will be testing to param_grid (Martin)
param_grid = {
    "activation"
}

keras_model = KerasRegressor(build_fn=create_model, verbose=0)

# TODO: for GridSearchCV modify any other params that we need (scoring, cv, num_jobs, etc) (Martin)
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid)
grid_result = grid.fit(X_train_scaled, y_train)

# TODO: implement model evaluation and print/graph anything needed for visualization

ModuleNotFoundError: No module named 'keras.wrappers'